#### **Thermodynamic Modeling & Design Optimization of a Solar-Driven $CO_2$ Reboiler**

#### **Purpose:** : This project utilizes the TESPy (Thermal Engineering Systems in Python) library to design and validate the thermodynamic steady-state of a 5 MW CSP-powered reboiler for carbon capture applications. The model simulates a counter-current heat exchanger where pressurized Solar Thermal Fluid (STF) delivers energy to an amine solution, facilitating the stripping of $CO_2$.


#### **Sources:** 
- Abu-Zahra et al. (2007): "CO2 capture from power plants: Part I. A parametric study of the technical performance based on monoethanolamine (MEA)."
- IEAGHG (2012): "Operating flexibility of power plants with tertiary solvent-based post-combustion capture."


##### **Author:** Bello Oluwatobi

##### **Last Updated:** November 3, 2025

### #1 Importing the libraries/modules

In [ ]:
# importing necessary modules from tespy
from tespy.networks import Network
from tespy.components import (
    Source, Sink, HeatExchanger
)
from tespy.connections import Connection

### #2 Network and Components Setup

In [ ]:
# using water as a substitute for amine solution for simplicity
nw = Network(fluids=['water'])

# setting units for the network
nw.units.set_defaults(temperature='°C', pressure='bar', enthalpy='kJ / kg', mass_flow='kg / s')

In [ ]:

# setting up components for the solar thermal supply
solar_source = Source('Solar Thermal Fluid Supply')
solar_sink   = Sink('Solar Thermal Fluid Return')

# setting up components for the amine solution loop
amine_source = Source('CO2 Rich Amine Inlet')
amine_sink   = Sink('Lean Amine Vapor-Liquid Outlet')

# setting up the solvent reboiler (counter-current heat exchanger)
reboiler = HeatExchanger('Solar Reboiler')

In [ ]:
# setting up connection: solar thermal supply -> reboiler
c1 = Connection(solar_source, 'out1', reboiler, 'in1', label='Solar_In')
# setting up connection: reboiler -> solar thermal return
c2 = Connection(reboiler, 'out1', solar_sink, 'in1', label='Solar_Out')

# setting up connection: amine solution inlet -> reboiler
c3 = Connection(amine_source, 'out1', reboiler, 'in2', label='Amine_In')
# setting up connection: reboiler -> amine solution outlet (back to process)
c4 = Connection(reboiler, 'out2', amine_sink, 'in1', label='Amine_Out')

# adding all connections to the network
nw.add_conns(c1, c2, c3, c4)

### #3 Setting System Design Parameters

In [ ]:
# setting the target heat transfer duty
TARGET_Q = 5e6  # 5 MW

# setting process temperature
T_PROCESS = 120 # °C

# setting the pressure drop to 2% at amine and solar fluid sides across the reboiler
reboiler.set_attr(
    Q=-TARGET_Q,
    pr1=0.98, 
    pr2=0.98)

# setting attributes for the amine inlet into the reboiler
c3.set_attr(
    T=120,              # saturation temp of liquid
    x=0,                # inlet to reboiler is liquid
    fluid={'water': 1} # using water as a substitute to amine solution
)

# setting attributes for the amine outlet from the reboiler
c4.set_attr(
    x=0.1               # the amine outlet is 10% vapor
)

# setting attributes for the solar thermal fluid to reboiler
c1.set_attr(
    T=140,              # fluid enters at 140°C
    p=10,               # 10 bar
    fluid={'water': 1}
)

# setting attributes for the solar thermal fluid leaving the reboiler
c2.set_attr(
    T=125               # Solar leaves at 125°C (Design Delta T = 15K)
)

### #4 Solving the System Network

In [ ]:
# solving the network in design mode
nw.solve(mode='design')

nw.print_results()

### #5 System Network Design Results

In [ ]:
# validating the results
actual_Q = c1.m.val_SI * (c1.h.val_SI - c2.h.val_SI)
print('-'*60)
print(f"Validation")
print('-'*60)
print(f"Target Duty: {TARGET_Q/1e6} MW")
print(f"Actual Duty: {actual_Q/1e6:.4f} MW")
print(f"Amine Mass Flow Required: {c3.m.val:.4f} kg/s")
print(f"Solar Thermal Fluid Mass Flow Required: {c1.m.val:.4f} kg/s")
print('-'*60)